In [1]:
pwd

'/Users/edwardterrell/Desktop/Training/kindle/strava'

In [2]:
import requests
import json
import time
import os
import shutil

from pprint import pprint
from datetime import date,timedelta,datetime

import pandas as pd
pd.set_option('display.precision', 1)
import warnings
warnings.filterwarnings("ignore", message=".*will drop timezone.*")

%load_ext autoreload
%autoreload 2
from clean_convert import cleanup, convert_speed, order_columns, process_new_data, save_data, get_date, get_year, remove_backups
from zones import get_zones_for_id, build_week_summary
from plots_details import display_detail_week

# import graphics programs
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(14,6)}) # for all future plots

import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/mlml/bin/python'

In [3]:
%matplotlib inline
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
from token_manager import get_valid_access_token
import requests

access_token = get_valid_access_token()
print("Using access token:", access_token)

# Make an authenticated API call
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)
print(response.json())

Access token expired. Refreshing...
Using access token: 2da75bd5a02156550ab543ea8702915b7db0861c
{'id': 723727, 'username': 'eterrell', 'resource_state': 2, 'firstname': 'Ed', 'lastname': 'Terrell', 'bio': '', 'city': 'Boulder', 'state': 'CO', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2012-07-05T00:04:09Z', 'updated_at': '2025-07-09T12:38:59Z', 'badge_type_id': 1, 'weight': 61.235, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/large.jpg', 'friend': None, 'follower': None}


### Load dataframes

In [5]:
# Loading empty dfs for user to start from scratch
strava_df = pd.read_pickle(f"data/strava_data_empty.pkl")
strava_zone_df = pd.read_pickle(f"data/strava_zone_data_empty.pkl")
!tree data

data
├── 2025-07-18
│   ├── strava_data_2024.pkl
│   └── strava_zone_data_2024.pkl
├── 2025-07-30
│   ├── strava_data_2025.pkl
│   └── strava_zone_data_2025.pkl
├── strava_data_2024.pkl
├── strava_data_2025.pkl
├── strava_data_empty.pkl
├── strava_zone_data_2024.pkl
├── strava_zone_data_2025.pkl
└── strava_zone_data_empty.pkl

3 directories, 10 files


### Select date range and year

In [6]:
# select 'empty' to start new dfs from scratch
# Default values
today = date.today().strftime('%Y-%m-%d')
yesterday = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')
tomorrow = (date.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# Get folder name and year from user
my_folder = get_date("Enter folder date or empty for None", default=yesterday)
print(f"✅ Folder selected: {my_folder}")
if my_folder is None:
    print("📭 Empty data mode selected. Skipping further input.")
else:
    # Get year from user
    year = get_year("Enter year", default='2025')
    print(f"✅ Year selected: {year}")
    # Reload dfs and confirm lengths
    strava_df = pd.read_pickle(f"data/{my_folder}/strava_data_{year}.pkl")
    strava_zone_df = pd.read_pickle(f"data/{my_folder}/strava_zone_data_{year}.pkl")

Enter folder date or empty for None [2025-07-31]:  2025-07-30


✅ Folder selected: 2025-07-30


Enter year [2025]:  


✅ Year selected: 2025


In [7]:
# confirm lengths and assign starting length to length_df for later comparision
length_df = len(strava_df)
print (f'There are {len(strava_df)} activities in strava_df.')
print (f'There are {len(strava_zone_df)} activities in strava_zone_df.')

There are 133 activities in strava_df.
There are 133 activities in strava_zone_df.


In [8]:
strava_df.head(1)
strava_zone_df.head(1)

,id,name,date,sport,distance,time,speed,elev_gain,avg_HR,max_HR,suffer_score,time-minutes,average_speed,start_date
0,15287217614,Walden Ponds - first time out since hamstring ...,2025-07-30,Run,3.2,0:34,10:39,28.2,152.8,183.0,9.0,34.0,2.5,2025-07-30 13:38:47+00:00


,id,name,date,suffer_score,start_date,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,intense,week_start,HR-value,weekly_intense,weekly_moderate,weekly_suffer_score,weekly_HR-value
0,15287217614,Walden Ponds - first time out since hamstring ...,2025-07-30,9.0,2025-07-30 13:38:47+00:00,1.7e-02,26.7,0.2,7.1,0.0,26.7,7.3,2025-07-28,20.6,35.8,351.6,89.0,137.3


### Update a specific activity

### Recent activities only!

In [9]:
# Get start and end dates from user
start_date_str = get_date("Enter START date", default=yesterday)
end_date_str = get_date("Enter END date", default=tomorrow)

print(f"\n✅ Date range selected:\nStart: {start_date_str}\nEnd:   {end_date_str}")


Enter START date [2025-07-31]:  
Enter END date [2025-08-02]:  



✅ Date range selected:
Start: 2025-07-31
End:   2025-08-02


In [10]:
# Desired number of activities for upload
num_of_activities_for_upload = 50 #select maximum desired

# change date to unix integer format with mktime
start_date = int(time.mktime(time.strptime(start_date_str, '%Y-%m-%d')))
end_date = int(time.mktime(time.strptime(end_date_str, '%Y-%m-%d')))
print()

# strava url
url = "https://www.strava.com/api/v3/athlete/activities"
headers = {"Authorization": f"Bearer {access_token}"}
params = {
    "before": end_date,
    "after": start_date,
    "per_page": 200,  # max per request
    "page": 1
}
activities = []

stop_fetching = False
while True:
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    if not data or stop_fetching:
        # breaks out of while loop
        break
        
    for activity in data:
        # change date to unix integer format
        activity_date = int(datetime.strptime(activity['start_date'], "%Y-%m-%dT%H:%M:%SZ").timestamp())
        print(f"ID: {activity['id']}, Name: {activity['name']}, Date: {activity['start_date']}")
        # set number of activites (strava maximum is 100/15 minutes)
        if len(activities) >= num_of_activities_for_upload:
            stop_fetching = True
            # breaks out of for loop only
            break
        activities.append(activity)
    params["page"] += 1


ID: 15310444923, Name: Logan Mill, Date: 2025-08-01T13:35:02Z


In [11]:
# Build out strava_df with only the necessary columns
recent_df = pd.DataFrame(activities)
cols_needed = ['id','name', 'distance', 'moving_time','total_elevation_gain', 'sport_type',
          'average_speed', 'average_heartrate','max_heartrate', 'suffer_score','start_date']
recent_df = recent_df.loc[:,cols_needed]
recent_df = recent_df.sort_values('start_date',ascending=False) 

# This step does the comparison between recent_df and strava_df
# strava_df holds previously saved data

if strava_df.empty:
    new_activities_df = recent_df.copy()
else:
    new_activities_df = recent_df[~recent_df.id.isin(strava_df.id)].copy()
    
# display number of new activities
print(f'You have {len(new_activities_df)} activity to process')


You have 1 activity to process


In [12]:
strava_check = process_new_data(new_activities_df, strava_df)

5.31
we are in convert_speed


In [13]:
if strava_check is None:
    print('strava_df not updated')
else:
    strava_df = strava_check

In [14]:
print (f'You haved processed {len(strava_df)- length_df} activities. There are now {len(strava_df)} activities in strava_df.')

You haved processed 1 activities. There are now 134 activities in strava_df.


### Get Zone data

In [15]:
# Recent zone data
# Create a new_zone dataframe for recent activities
cols = ['id', 'name', 'date', 'suffer_score','start_date']
new_data = strava_df.loc[:,cols]
new_zone_df = pd.DataFrame(new_data, columns=cols)

mask = new_zone_df['id'].isin(new_activities_df['id'])
new_zone_df = new_zone_df[mask]

In [16]:
new_zone_df

,id,name,date,suffer_score,start_date
0,15310444923,Logan Mill,2025-08-01,69.0,2025-08-01 13:35:02+00:00


In [17]:
# get_zones_for_id is a function in zones.py
# There is a max limit of 100 calls per 15 minutes set by Strava for this function
    
if len(new_zone_df) > 0 and len(new_zone_df) < 100:
    print (len(new_zone_df))
    # Apply function to fetch zone data per activity ID
    zone = new_zone_df['id'].apply(lambda current_id: get_zones_for_id(current_id, access_token))
    zone.index = new_zone_df['id']

    # display new zone info
    print()
    zone

elif  len(new_zone_df)==0:
    print ("No new activites to process")
    
elif len(new_zone_df) >= 100:
     print("❌ Number of calls will exceed Strava's 15-minute limit (max 100)")
     print ("Try limiting the number of calls before proceeding")

1
Fetching zones for activity: 15310444923



,Zone1,Zone2,Zone3,Zone4,Zone5
id,,,,,
15310444923,83.5,64.5,26.4,24.0,0.2


In [18]:
if len(new_zone_df) > 0 and len(new_zone_df) < 100:
    new_zone_df = pd.merge(new_zone_df, zone, left_on='id',right_index=True)

    # Get the Sunday start of the week for each date (time set to Sunday at midnight)
    new_zone_df['week_start'] = new_zone_df['start_date'].dt.to_period('W-SUN').dt.start_time
    
    #new_zone_df.columns are now matched with strava_zone_df
    new_zone_df = pd.concat ([new_zone_df, strava_zone_df], axis=0)
    
    len(strava_zone_df)
    len(new_zone_df)
    
    # Correct instance of unreliable heart rate data; adds erroneous zone5 time to zone2
    mask = new_zone_df['Zone5'] > 5
    new_zone_df.loc[mask, 'Zone2'] = new_zone_df.loc[mask, 'Zone2'] + new_zone_df.loc[mask, 'Zone5']
    new_zone_df.loc[new_zone_df.suffer_score < 100, 'Zone5'] = 0
    
    # Add in summary zones: moderate and intense
    new_zone_df['moderate'] = new_zone_df[['Zone1', 'Zone2']].sum(axis=1)
    new_zone_df['intense'] = new_zone_df[['Zone3', 'Zone4', 'Zone5']].sum(axis=1)
    
    new_zone_df['HR-value'] = (new_zone_df.Zone2*0.5)+(new_zone_df.intense)

    # This drops all weekly_ cols since they are all recalculated later
    only_cols = ['id', 'name', 'date', 'suffer_score', 'start_date', 'Zone1', 'Zone2',
           'Zone3', 'Zone4', 'Zone5', 'moderate', 'intense', 'week_start','HR-value']
    new_zone_df = new_zone_df[only_cols]
    
    
    # Temporarily sort ascending by week_start and date to compute cumulative sum correctly
    temp = new_zone_df.sort_values(['week_start', 'start_date'])

    # Calculate cumulative sum within each week for weekly_ cols
    cumulative_cols = ['intense', 'moderate', 'suffer_score','HR-value']
    cumulative_sums = (
    temp
        .groupby('week_start')[cumulative_cols]
        .cumsum()
        .add_prefix('weekly_')  # rename columns if needed
    )
    # Add start_date column for reference
    cumulative_sums['start_date'] = temp['start_date']
    cumulative_sums = cumulative_sums.sort_values('start_date', ascending = False)
    cumulative_sums.head(3)
    strava_zone_df = pd.merge(new_zone_df,cumulative_sums, left_on ='start_date', right_on='start_date')

133

134

,weekly_intense,weekly_moderate,weekly_suffer_score,weekly_HR-value,start_date
0,86.1,499.6,158.0,219.9,2025-08-01 13:35:02+00:00
0,35.8,351.6,89.0,137.3,2025-07-30 13:38:47+00:00
1,28.5,324.9,80.0,116.7,2025-07-29 14:18:57+00:00


In [19]:
strava_zone_df.head()

,id,name,date,suffer_score,start_date,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,intense,week_start,HR-value,weekly_intense,weekly_moderate,weekly_suffer_score,weekly_HR-value
0,15310444923,Logan Mill,2025-08-01,69.0,2025-08-01 13:35:02+00:00,8.3e+01,64.5,26.4,24.0,0.0,148.0,50.3,2025-07-28,82.6,86.1,499.6,158.0,219.9
1,15287217614,Walden Ponds - first time out since hamstring ...,2025-07-30,9.0,2025-07-30 13:38:47+00:00,1.7e-02,26.7,0.2,7.1,0.0,26.7,7.3,2025-07-28,20.6,35.8,351.6,89.0,137.3
2,15278734733,"Gravel, Gravel, Gravel: Coal Creek and Rock Creek",2025-07-29,47.0,2025-07-29 14:18:57+00:00,1.3e+02,100.9,8.7,0.0,0.0,233.2,8.7,2025-07-28,59.1,28.5,324.9,80.0,116.7
3,15263842880,Old Stage and gravel roads,2025-07-28,33.0,2025-07-28 12:30:16+00:00,1.6e+01,75.5,14.2,5.7,0.0,91.7,19.8,2025-07-28,57.6,19.8,91.7,33.0,57.6
4,15255996830,Gold Hill via Gold Run,2025-07-27,82.0,2025-07-27 13:40:08+00:00,4.9e+01,80.7,68.6,12.7,0.0,130.1,81.3,2025-07-21,121.7,292.4,558.3,342.0,484.7


### Plotting Section

In [20]:
# Reccomended weekly maximum of 540 minutes
week_summary = build_week_summary(strava_zone_df)
week_summary.sort_values('week_start',ascending=False).head()

,week_start,weekly_intense,weekly_moderate,weekly_suffer_score,weekly_HR-value,week
29,2025-07-28,86.1,499.6,158.0,219.9,Jul 28
28,2025-07-21,292.4,558.3,342.0,484.7,Jul 21
27,2025-07-14,122.0,704.2,538.0,461.5,Jul 14
26,2025-07-07,152.0,767.9,546.0,495.0,Jul 07
25,2025-06-30,143.9,794.7,597.0,515.3,Jun 30


In [21]:
ax = week_summary.plot(
    y=['weekly_moderate','weekly_intense'],
    x='week',
    kind='bar', 
    stacked=True, 
    figsize=(14, 6), 
    color=['#98FB98', '#F08080']
)

# Overlay line plot for 'weekly_suffer_score'
line = ax.plot(
    week_summary["week"],
    week_summary["weekly_HR-value"],
    color="black",
    marker="o",
    linewidth=2,
    label="HR-value"  
)

# Overlay line plot for 'weekly_suffer_score'
line2 = ax.plot(
    week_summary["week"],
    week_summary["weekly_suffer_score"],
    color="grey",
    marker=None,
    linewidth=1,
    label="Suffer_Score"  
)

plt.axhline(y=540, color='red', linestyle='--', linewidth=1)

plt.title("Time in Heart Rate Zones");
plt.xlabel("Activity ID");
plt.ylabel("Time (minutes)");
plt.legend(title="Zone");
plt.xticks(rotation=45, ha='right');
plt.tight_layout();
plt.show();

### Drilling down a specific week

In [22]:
# default start is current week
start = week_summary.sort_values('week_start',ascending=False).iloc[0,0]
#start = "2025-06-02" # <-- select custom week starting date

In [23]:
week_df, zone_week_df = display_detail_week(start, strava_df, strava_zone_df)

In [24]:
week_df.sort_values('start_date',ascending=False)[['id','name', 'date', 'sport', 'distance', 'time', 'speed', 'elev_gain',
       'avg_HR', 'max_HR', 'suffer_score']]
zone_week_df.sort_values('start_date',ascending=False)[['name','date','Zone1', 'Zone2',
       'Zone3', 'Zone4', 'Zone5', 'moderate', 'intense','HR-value','suffer_score','week_start','weekly_moderate',
       'weekly_intense','weekly_HR-value']]

,id,name,date,sport,distance,time,speed,elev_gain,avg_HR,max_HR,suffer_score
0,15310444923,Logan Mill,2025-08-01,Ride,39.3,3:18,11.9,2872.0,108.5,148.0,69.0
0,15287217614,Walden Ponds - first time out since hamstring ...,2025-07-30,Run,3.2,0:34,10:39,28.2,152.8,183.0,9.0
0,15278734733,"Gravel, Gravel, Gravel: Coal Creek and Rock Creek",2025-07-29,Ride,49.4,4:01,12.3,1820.9,100.9,129.0,47.0
1,15263842880,Old Stage and gravel roads,2025-07-28,Ride,24.1,1:51,13.0,1315.6,110.6,140.0,33.0


,name,date,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,intense,HR-value,suffer_score,week_start,weekly_moderate,weekly_intense,weekly_HR-value
0,Logan Mill,2025-08-01,8.3e+01,64.5,26.4,24.0,0.0,148.0,50.3,82.6,69.0,2025-07-28,499.6,86.1,219.9
1,Walden Ponds - first time out since hamstring ...,2025-07-30,1.7e-02,26.7,0.2,7.1,0.0,26.7,7.3,20.6,9.0,2025-07-28,351.6,35.8,137.3
2,"Gravel, Gravel, Gravel: Coal Creek and Rock Creek",2025-07-29,1.3e+02,100.9,8.7,0.0,0.0,233.2,8.7,59.1,47.0,2025-07-28,324.9,28.5,116.7
3,Old Stage and gravel roads,2025-07-28,1.6e+01,75.5,14.2,5.7,0.0,91.7,19.8,57.6,33.0,2025-07-28,91.7,19.8,57.6


### Saving data

In [25]:
strava_df = strava_df.sort_values('id',ascending=False)
strava_zone_df = strava_zone_df.sort_values('id',ascending=False)

In [26]:
# verify before saving
strava_df.head(1)
strava_zone_df.head(1)
print(len(strava_df))
print(len(strava_zone_df))

,id,name,date,sport,distance,time,speed,elev_gain,avg_HR,max_HR,suffer_score,time-minutes,average_speed,start_date
0,15310444923,Logan Mill,2025-08-01,Ride,39.3,3:18,11.9,2872.0,108.5,148.0,69.0,198.6,5.3,2025-08-01 13:35:02+00:00


,id,name,date,suffer_score,start_date,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,intense,week_start,HR-value,weekly_intense,weekly_moderate,weekly_suffer_score,weekly_HR-value
0,15310444923,Logan Mill,2025-08-01,69.0,2025-08-01 13:35:02+00:00,83.5,64.5,26.4,24.0,0.0,148.0,50.3,2025-07-28,82.6,86.1,499.6,158.0,219.9


134
134


### Use save_data, and year

In [27]:
# Select year for saving
# Data will be saved to: data/{today}
save_data(strava_df, strava_zone_df, 2025)

Data saved in folder: data/2025-08-01


### Remove older backup files

In [31]:
!tree data

data
├── 2025-07-18
│   ├── strava_data_2024.pkl
│   └── strava_zone_data_2024.pkl
├── 2025-07-30
│   ├── strava_data_2025.pkl
│   └── strava_zone_data_2025.pkl
├── 2025-08-01
│   ├── strava_data_2025.pkl
│   └── strava_zone_data_2025.pkl
├── strava_data_2024.pkl
├── strava_data_2025.pkl
├── strava_data_empty.pkl
├── strava_zone_data_2024.pkl
├── strava_zone_data_2025.pkl
└── strava_zone_data_empty.pkl

4 directories, 12 files


In [32]:
# def remove_other_backups(base_dir="data", keep_subdirs=None, verbose=True):
remove_backups(base_dir="data", keep_subdirs={today})

Deleting: data/2025-07-30/strava_data_2025.pkl
Deleting: data/2025-07-30/strava_zone_data_2025.pkl
Removing empty directory: data/2025-07-30


### Other info

In [ ]:
# Set number of rows to display
pd.set_option('display.max_rows', 40)  # if set to None this will show all rows
pd.set_option('display.min_rows', 40)  # adjust to help override auto-truncation

strava_zone_df[['name','date','moderate','intense','HR-value','suffer_score']]
pd.reset_option('display.max_rows')